# PodmanPy: 用于 Podman 服务的 Python 脚本

PodmanPy 是 Python3 模块，允许您编写 Python 脚本来访问由 Podman 服务维护的资源。它利用了 Podman 服务的 RESTful API。

Podman 服务通过 URL 进行访问，其中协议表明客户端如何连接到服务。支持的协议有：`http+ssh`、`http+unix` 或 `tcp`。格式如下：
- `http+ssh://[<login>@]<hostname>[:<port>]/<full filesystem path>`
    - `http+ssh://alice@api.example:22/run/user/1000/podman/podman.sock`
    - 协议(scheme) `ssh` 被视为别名
- `http+unix://<full filesystem path>`
    - `http+unix:///run/podman/podman.sock`- 
    - 协议(scheme) `unix` 被视为别名
- `tcp://<hostname>:<port>`
    - `tcp://api.example:8888`


安装：
```bash
pip install podman
```

```python
import json
from podman import PodmanClient

# Provide a URI path for the libpod service.  In libpod, the URI can be a unix
# domain socket(UDS) or TCP.  The TCP connection has not been implemented in this
# package yet.

uri = "unix:///run/user/1000/podman/podman.sock"

with PodmanClient(base_url=uri) as client:
    version = client.version()
    print("Release: ", version["Version"])
    print("Compatible API: ", version["ApiVersion"])
    print("Podman API: ", version["Components"][0]["Details"]["APIVersion"], "\n")

    # get all images
    for image in client.images.list():
        print(image, image.id, "\n")

    # find all containers
    for container in client.containers.list():
        # After a list call you would probably want to reload the container
        # to get the information about the variables such as status.
        # Note that list() ignores the sparse option and assumes True by default.
        container.reload()
        print(container, container.id, "\n")
        print(container, container.status, "\n")

        # available fields
        print(sorted(container.attrs.keys()))

    print(json.dumps(client.df(), indent=4))
```